In [4]:
import sympy as sp

# Define functions for individual transformations
def Rx(alpha):
  """
  This function creates a symbolic rotation matrix around the X-axis.

  Args:
      alpha (sp.Symbol): Rotation angle.

  Returns:
      sp.Matrix: The symbolic rotation matrix.
  """
  ca = sp.cos(alpha)
  sa = sp.sin(alpha)
  return sp.Matrix([[1, 0, 0, 0],
                   [0, ca, -sa, 0],
                   [0, sa, ca, 0],
                   [0, 0, 0, 1]])

def Tx(a):
  """
  This function creates a symbolic translation matrix along the X-axis.

  Args:
      a (sp.Symbol): Translation distance.

  Returns:
      sp.Matrix: The symbolic translation matrix.
  """
  return sp.Matrix([[1, 0, 0, a],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])

def Rz(theta):
  """
  This function creates a symbolic rotation matrix around the Z-axis.

  Args:
      theta (sp.Symbol): Rotation angle.

  Returns:
      sp.Matrix: The symbolic rotation matrix.
  """
  ca = sp.cos(theta)
  sa = sp.sin(theta)
  return sp.Matrix([[ca, -sa, 0, 0],
                   [sa, ca, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])

def Tz(d):
  """
  This function creates a symbolic translation matrix along the Z-axis.

  Args:
      d (sp.Symbol): Translation distance.

  Returns:
      sp.Matrix: The symbolic translation matrix.
  """
  return sp.Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, d],
                   [0, 0, 0, 1]])

# Function for combined transformation
def DH(alpha, a, d, theta):
  """
  This function builds a symbolic transformation matrix using the Denavit-Hartenberg (DH) convention.

  Args:
      alpha (sp.Symbol): Twist angle about the previous z-axis.
      a (sp.Symbol): Offset distance along the previous z-axis.
      theta (sp.Symbol): Rotation angle about the current x-axis.
      d (sp.Symbol): Offset distance along the current x-axis.

  Returns:
      sp.Matrix: The symbolic transformation matrix.
  """

  # Define the rotation and translation components
  R_alpha = Rx(alpha)
  R_theta = Rz(theta)
  T_d = Tz(d)
  T_a = Tx(a)

  # Apply the transformations in the DH convention order
  T = R_alpha * T_a * R_theta * T_d

  return T

def get_T_from_DH(DH_table, all_transforms=False):

    # Get the symbolic output
    T_01 = DH(alpha_0, a_0, d_1, theta_1)
    T_12 = DH(alpha_1, a_1, d_2, theta_2)
    T_23 = DH(alpha_2, a_2, d_3, theta_3)
    T_34 = DH(alpha_3, a_3, d_4, theta_4)
    T_45 = DH(alpha_4, a_4, d_5, theta_5)

    T_01 = T_01.subs(D_H_table)
    T_12 = T_12.subs(D_H_table)
    T_23 = T_23.subs(D_H_table)
    T_34 = T_34.subs(D_H_table)
    T_45 = T_45.subs(D_H_table)

    T_04 = sp.simplify(T_01 * T_12 * T_23 * T_34)
    T_05 = sp.simplify(T_01 * T_12 * T_23 * T_34 * T_45)

    if all_transforms:
        return T_01, T_12, T_23, T_34, T_45, T_04, T_05
    return T_04

# # set the values of the symbolic variables
# T_01 = T_01.subs({alpha_0: 0, a_0: 0, d_1: L_1, theta_1: theta_1})
# T_12 = T_12.subs({alpha_1: 0, a_1: L_2, d_2: 0, theta_2: theta_2})
# T_23 = T_23.subs({alpha_2: sp.pi, a_2: L_3, d_3: L_4+d_3, theta_3: 0})
# T_34 = T_34.subs({alpha_3: 0, a_3: 0, d_4: 0, theta_4: theta_4})
# T_45 = T_45.subs({alpha_4: 0, a_4: 0, d_5: L_5, theta_5: 0})

# Define symbolic variables
alpha_0, a_0, d_1, theta_1 = sp.symbols('alpha_0 a_0 d_1 theta_1')
alpha_1, a_1, d_2, theta_2 = sp.symbols('alpha_1 a_1 d_2 theta_2')
alpha_2, a_2, d_3, theta_3 = sp.symbols('alpha_2 a_2 d_3 theta_3')
alpha_3, a_3, d_4, theta_4 = sp.symbols('alpha_3 a_3 d_4 theta_4')
alpha_4, a_4, d_5, theta_5 = sp.symbols('alpha_4 a_4 d_5 theta_5')

L_0, L_1, L_2, L_3, L_4, L_5 = sp.symbols('L_0 L_1 L_2 L_3 L_4 L_5')

In [5]:
D_H_table = {
    alpha_0: 0, a_0: 0, d_1: L_0+L_1, theta_1: theta_1,
    alpha_1: 0, a_1: L_2, d_2: 0, theta_2: theta_2,
    alpha_2: sp.pi, a_2: L_3, d_3: L_4+d_3, theta_3: 0,
    alpha_3: 0, a_3: 0, d_4: 0, theta_4: theta_4,
    alpha_4: 0, a_4: 0, d_5: L_5, theta_5: 0
}

T_04 = get_T_from_DH(D_H_table)

# sp.simplify(T_01 * sp.simplify(T_12 * sp.simplify(T_23 * T_34)))
display(T_04)

Matrix([
[cos(theta_1 + theta_2 - theta_4),  sin(theta_1 + theta_2 - theta_4),  0, L_2*cos(theta_1) + L_3*cos(theta_1 + theta_2)],
[sin(theta_1 + theta_2 - theta_4), -cos(theta_1 + theta_2 - theta_4),  0, L_2*sin(theta_1) + L_3*sin(theta_1 + theta_2)],
[                               0,                                 0, -1,                         L_0 + L_1 - L_4 - d_3],
[                               0,                                 0,  0,                                             1]])

In [6]:
D_H_table = {
    alpha_0: 0, a_0: 0, d_1: 0, theta_1: theta_1,
    alpha_1: 0, a_1: L_2, d_2: L_0+L_1, theta_2: theta_2,
    alpha_2: sp.pi, a_2: L_3, d_3: L_4+d_3, theta_3: 0,
    alpha_3: 0, a_3: 0, d_4: 0, theta_4: theta_4,
    alpha_4: 0, a_4: 0, d_5: L_5, theta_5: 0
}

T_04 = get_T_from_DH(D_H_table)

# sp.simplify(T_01 * sp.simplify(T_12 * sp.simplify(T_23 * T_34)))
display(T_04)

Matrix([
[cos(theta_1 + theta_2 - theta_4),  sin(theta_1 + theta_2 - theta_4),  0, L_2*cos(theta_1) + L_3*cos(theta_1 + theta_2)],
[sin(theta_1 + theta_2 - theta_4), -cos(theta_1 + theta_2 - theta_4),  0, L_2*sin(theta_1) + L_3*sin(theta_1 + theta_2)],
[                               0,                                 0, -1,                         L_0 + L_1 - L_4 - d_3],
[                               0,                                 0,  0,                                             1]])

In [9]:
print(sp.simplify(T_04))

Matrix([[cos(theta_1 + theta_2 - theta_4), sin(theta_1 + theta_2 - theta_4), 0, L_2*cos(theta_1) + L_3*cos(theta_1 + theta_2)], [sin(theta_1 + theta_2 - theta_4), -cos(theta_1 + theta_2 - theta_4), 0, L_2*sin(theta_1) + L_3*sin(theta_1 + theta_2)], [0, 0, -1, L_0 + L_1 - L_4 - d_3], [0, 0, 0, 1]])
